In [43]:
# %cd ..
%cd /home/janneke/Documents/Master/Machine_Learning_in_Practice/HMS/MLiP_group_10_task1_HMS/

/home/janneke/Documents/Master/Machine_Learning_in_Practice/HMS/MLiP_group_10_task1_HMS


In [44]:
from copy import deepcopy

import pandas as pd
import numpy as np
from logging import getLogger, basicConfig, INFO
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
import pickle

from datasets.raw_data_loader import CustomRawDataset
from datasets.data_loader_configs import BaseDataConfig
from utils.evaluation_utils import score_kl_divergence

In [45]:
basicConfig(level=INFO)
logger = getLogger("main")
config = BaseDataConfig()

In [46]:
subset_sample_count = 0
dataset = CustomRawDataset(
    config, mode="train", cache=True, subset_sample_count=subset_sample_count
)
dataset.print_summary()

2024-02-17 13:35:36,166 - data_loader.log - INFO - Loading dataset from cache: ./data/cache/CustomRawDataset_0_train.npz
2024-02-17 13:35:36,166 - data_loader.log - INFO - Loading dataset from cache: ./data/cache/CustomRawDataset_0_train.npz
2024-02-17 13:35:36,166 - data_loader.log - INFO - Loading dataset from cache: ./data/cache/CustomRawDataset_0_train.npz
2024-02-17 13:35:36,166 - data_loader.log - INFO - Loading dataset from cache: ./data/cache/CustomRawDataset_0_train.npz
2024-02-17 13:35:36,166 - data_loader.log - INFO - Loading dataset from cache: ./data/cache/CustomRawDataset_0_train.npz
2024-02-17 13:35:36,166 - data_loader.log - INFO - Loading dataset from cache: ./data/cache/CustomRawDataset_0_train.npz
INFO:data_loader.log:Loading dataset from cache: ./data/cache/CustomRawDataset_0_train.npz


Dataset Summary:
Mode: train
Total Samples: 106799
Unique Patients: 1950
Unique EEGs: 17088
Unique Spectrograms: 11138
Label Distribution:
seizure_vote     93773
lpd_vote        121622
gpd_vote        135094
lrda_vote       101278
grda_vote       113121
other_vote      209986
dtype: int64

Vote Statistics:
        seizure_vote  lpd_vote  gpd_vote  lrda_vote  grda_vote  other_vote
mean        0.878033  1.138793  1.264937   0.948305   1.059195    1.966179
median      0.000000  0.000000  0.000000   0.000000   0.000000    0.000000
var         2.368146  7.945951  9.808804   4.565943   4.966214   13.111929
Probabilities Loaded: 106799
Features Loaded: 106799

Configuration Summary:
+---------------------+--------------------------------------------------------------------------------------------------------------------------------------+
| Configuration       | Value                                                                                                                               

In [47]:
x_train, x_test, y_train, y_test = train_test_split(
    dataset.features_per_sample,
    dataset.lbl_probabilities,
    test_size=0.33,
    random_state=42,
)
y0, y1, y2, y3, y4, y5 = zip(*y_train)

Let multiple separate models regress on one label each


In [48]:
y_pred = np.zeros(y_test.shape)

models = {}
for i, lbl_group in enumerate([y0, y1, y2, y3, y4, y5]):
    y_train_group = lbl_group
    clf = GradientBoostingRegressor(
        n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0
    ).fit(x_train, y_train_group)
    clf.fit(x_train, y_train_group)
    y_pred_group = clf.predict(x_test)
    y_pred[:, i] = y_pred_group
    models[lbl_group] = clf

y_pred[y_pred < 0] = 0
y_pred_probabilities = y_pred / np.sum(y_pred, axis=1)[:, None]

In [49]:
with open(
    f"checkpoints/other_models/ensemble_one_model_per_target_{subset_sample_count}.pickle",
    "wb",
) as pickle_file:
    pickle.dump(models, pickle_file)

In [50]:
submission = pd.DataFrame(y_pred_probabilities, columns=dataset.label_cols)
solution = pd.DataFrame(y_test, columns=dataset.label_cols)

# Need to be added for score calculation, but are removed at the start of the function
submission.insert(0, "id", range(len(submission)))
solution.insert(0, "id", range(len(solution)))

score_kl_divergence(
    solution=deepcopy(solution),
    submission=deepcopy(submission),
    row_id_column_name="id",
)

1.529194744430282

In [51]:
# Random
random_pred = np.random.rand(33, 6)
random_pred[random_pred < 0] = 0
random_pred_probabilities = random_pred / np.sum(random_pred, axis=1)[:, None]
random_submission = pd.DataFrame(random_pred_probabilities, columns=dataset.label_cols)

random_submission.insert(0, "id", range(len(random_submission)))

score_kl_divergence(
    solution=deepcopy(solution),
    submission=deepcopy(random_submission),
    row_id_column_name="id",
)

0.0014772346517872553